<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/insurance_myV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import Counter

# Step 1: Load the dataset
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify file path
file_path = '/content/drive/My Drive/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
data = pd.read_csv(file_path)

# Step 2: Explore the data
print(data.head())
print(data.info())

# Check for missing values
print(data.isnull().sum())

# Display basic statistics
print(data.describe())

# Step 3: Visualize the distribution of the target variable (AMT_claim)
plt.figure(figsize=(10, 6))
sns.histplot(data['AMT_claim'], bins=50, kde=True)
plt.title('Distribution of AMT_claim')
plt.xlabel('Claim Amount')
plt.ylabel('Frequency')
plt.show()

# Step 4: Prepare the data for SMOTE
# Assuming we want to balance based on NB_claim (0 for no claim, 1 for claim)
X = data.drop(['NB_claim', 'AMT_claim'], axis=1)
y = data['NB_claim']

# Encode categorical variables (if any)
X = pd.get_dummies(X, drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Apply SMOTE
print("Class distribution before SMOTE:")
print(Counter(y_train))

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

print("Class distribution after SMOTE:")
print(Counter(y_train_resampled))

# Visualize the effect of SMOTE
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x=y_train)
plt.title('Class Distribution Before SMOTE')
plt.subplot(1, 2, 2)
sns.countplot(x=y_train_resampled)
plt.title('Class Distribution After SMOTE')
plt.tight_layout()
plt.show()